<a href="https://colab.research.google.com/gist/tolgakurtuluss/c20fa444ed04099d43f6dec0260df31a/bahcelievlerpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Building an AI Running Assistant to get Recommendations for optimum walking/running conditions using OpenCV, YOLOv2 and OpenWeatherMap**


> Main problem;

*   Problem here is to decide to go out for walking either my favourite walking trail is crowded by others and weather is optimal or not.
*   I used open-sourced streaming video source of Bahcelievler Belediyesi Kuleli live webcamera where it's accessible by everyone.
(Source: Bahçelievler Belediyesi Video Platformu - https://video.bahcelievler.bel.tr/)


> Step-by-step solutions;

*   I used **OpenCV** and a pre-trained "lightweight" model known for its speed that's still relatively accurate, **YOLOv2 in Darkflow** (the Tensorflow implementation of YOLO in Darknet) to count how many people are using the walking trail **per minute**.
*   Using OpenWeatherMap's one time call api, gathered **weather data** for exact location of park with walking trail.
*   Final decision is based on if **people per minute** is greater than **15 people** and **live feeling weather** is less than **5 °C**, it is not recommended to walk/run.



> Credits;

* (Faster video file FPS with cv2.VideoCapture and OpenCV) - https://www.pyimagesearch.com/2017/02/06/faster-video-file-fps-with-cv2-videocapture-and-opencv/
* (Computer Vision with EOL v3 Images) - https://github.com/aubricot/computer_vision_with_eol_images

In [1]:
# Install required libraries
!pip install opencv
!pip install tensorflow-gpu
!pip install imageio

# Download and build darkflow (the tensorflow implementation of YOLO)
import os
import pathlib

if "darkflow-master" in pathlib.Path.cwd().parts:
  while "darkflow-master" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path("darkflow-master").exists():
  !git clone --depth 1 https://github.com/thtrieu/darkflow.git
  # Compile darkflow
  %cd darkflow
  !python setup.py build_ext --inplace
  # Change darkflow to darkflow-master to distinguish between folder names
  %cd ../
  !mv darkflow darkflow-master
  %cd darkflow-master

ERROR: Could not find a version that satisfies the requirement opencv (from versions: none)
ERROR: No matching distribution found for opencv
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/usr/local/lib/python3.7/dist-packages/tensorflow_estimator-2.7.0.dist-info/METADATA'



In [2]:
# Upload yolo.weights, pre-trained weights file (for YOLO v2) from an external Google drive 
weights = 'yolo'
weights_file = weights + '.weights'
if not os.path.exists('weights_file'):
  !gdown --id 0B1tW_VtY7oniTnBYYWdqSHNGSUU
  !mkdir bin
  !mv yolo.weights bin

# Imports
%cd darkflow-master
%tensorflow_version 1.15.0rc2


Downloading...
From: https://drive.google.com/uc?id=0B1tW_VtY7oniTnBYYWdqSHNGSUU
To: /content/yolo.weights
100% 204M/204M [00:01<00:00, 187MB/s]
mkdir: cannot create directory ‘bin’: File exists
/content/darkflow-master
`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.15.0rc2`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


In [3]:
#Here TFNet pre-trained model is used for image detection with 0.5 treshold value.
!pip install tensorflow-estimator==1.15.1
import cv2
import os
import time
from darkflow.net.build import TFNet
from google.colab.patches import cv2_imshow

options = {"model": "/content/darkflow-master/cfg/yolo.cfg", "load": "/content/darkflow-master/bin/yolo.weights", "threshold": 0.5}

tfnet = TFNet(options)

     |████████████████████████████████| 503 kB 5.5 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/usr/local/lib/python3.7/dist-packages/tensorflow_estimator-2.7.0.dist-info/RECORD'

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.









Parsing ./cfg/yolo.cfg
Parsing /content/darkflow-master/cfg/yolo.cfg
Loading /content/darkflow-master/bin/yolo.weights ...
Successfully identified 203934260 bytes
Finished in 0.029663801193237305s
Model has a coco model name, loading coco labels.

Building net ...

Source | Train? | Layer 

In [4]:
## Counting how many person passes by the running trail by OpenCV Capture function 

timeout = time.time() + 60*5
finalresult = []

while True:
    if time.time() < timeout:
        vcap = cv2.VideoCapture("https://wowza.bahcelievler.bel.tr/live/kuleli6.stream/playlist.m3u8")
        ret, frame = vcap.read()
        frame = frame[200:355,130:250]
        results = tfnet.return_predict(frame)
        label_count = {}
        for result in results:
          label = result['label']
          if label in label_count:
            label_count[label] = label_count[label]+1
          else:
            label_count[label] = 1
          finalresult.append(label_count)
        print(label_count)
    else: 
        break

{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{'person': 1}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{'person': 2}
{'person': 2}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{}
{'person': 1}
{}
{}
{}
{}
{}
{}


In [5]:
import pandas as pd
df = pd.DataFrame(finalresult)
kisisayisi = int(df.sum()['person'] / 5)

#How many people we could detect per minute?
kisisayisi

2

In [6]:
## Using OpenWeather One-Call Api, we can gather weather data of our exact park location.

import requests

response = requests.get("https://openweathermap.org/data/2.5/onecall?lat=41.0023&lon=28.8598&units=metric&appid=439d4b804bc8187953eb36d2a8c26a02").json()['current']

weather = response['weather']
feelslike = response['feels_like'] 

In [7]:
## As a result of our project, we can understand whether we should go out for walking/running according to number of people and weather condition.

if (feelslike <= 5 or kisisayisi >= 15):
  print("❌ Bugün koşmamanı tavsiye ederim.","Dakikada",kisisayisi,"kişiyi tespit ettim ve hissedilen hava",feelslike,"°C.")
else:
  print("✅ Bugün koşmak için ortam müsait.","Dakikada",kisisayisi,"kişiyi tespit ettim ve hissedilen hava",feelslike,"°C.")

❌ Bugün koşmamanı tavsiye ederim. Dakikada 2 kişiyi tespit ettim ve hissedilen hava 0.39 °C.
